# Algoritmos Genéticos

#### Alunos: Arthur Bizon, Gabriel Schneider e Luciane Tedesco

Este trabalho discute a implementação do algoritmo genético baseado no caso do Caixeiro Viajante.

Assumimos que há um caixeiro-viajante que precisa visitar n cidades através do caminho mais curto
possível. Ele visita cada cidade exatamente uma vez, em seguida, retorna para a cidade onde ele
começou. Portanto, uma solução seria uma lista de todas as cidades na ordem que ele as visita: como
Cidade 1, Cidade 2,..., Cidade n; onde a cidade 1 é o seu estado inicial. Assim, os cromossomos para o
algoritmo genético serão diferentes permutações dos números inteiros de 1 a n.

<img src="Images/Alg-Geneticos.png">

Há muitas variações do problema caixeiro-viajante, mas para este trabalho faremos as seguintes
suposições:
- Cidades: Representadas por cromossomos, são números inteiros de 1 a 20.
- As n cidades serão listados na ordem c1 → c2 → ... → cn num dado cromossomo. 
- Coordenadas: Consideramos 20 cidades com coordenadas representadas por números aleatórios entre (0,0) e (1,1) em um plano cartesiano.
- Partindo do princípio que estamos tentando minimizar a distância total que o caixeiro viajante percorre, a função de aptdão será a distância total D:
<img src="Images/D1.png" width=300 align="left" >    


- A cidade n+1 = Cidade 1 (estado inicial). Se (xi, yi) são as coordenadas da cidade ci, logo, a função de aptidão (fitness) é dada pela distância euclidiana:
<img src="Images/D2.png" width=300 align="left"> 

- Assumimos que a distância d da cidade ci para a cidade cj é: d (ci, cj) = d (cj, ci) para todo i ∈
[1, n] e j ∈ [1, n]. 
- O tamanho da população é 20 e o número de interações é 10.000.
- Para construir as gerações subsequentes, mantemos a metade da população atual e geramos a outra metade da nova geração por meio da seleção e do crossover, mantendo apenas parte da população.
- Para determinar a distribuição do conjunto de cromossomos que serão escolhidos para a reprodução, calculamos a probabilidade, na qual distribuimos os valores inversamente (visto que organizamos os cromossomos em ordem crescente do mais apto ao menos apto).
- Para selecionar os cromossomo, utilizamos o método da roleta.
- Como no nosso caso o agente visita somente uma vez cada cidade, utilizaremos para fazer a Recombinação (crossover) a técnica de “cycle” (exemplo na Figura 1), onde:

        - Escolhemos um local aleatório dentro do cromossomo.
        
        - Passo 1: Os dois cromossomos pais trocam os números inteiros neste local para gerar os descendentes.
          A menos que os números trocados tenham o mesmo valor, cada descendente terá um número duplicado.
          
        - Passo 2: Em seguida, mudamos o número duplicado da primeira descendência com o mesmo local do número 
          da segunda descendência.
          
        - Passo 3 e 4: Isto significa que temos agora outro número duplicado então, repitiremos este processo 
          até não terem mais números duplicados.

Exemplo de Recombinação Cycle:
<img src="Images/Cycle.png"> 

- O operador de mutação escolhe aleatoriamente dois números inteiros em um cromossomo da nova geração e os troca. O operador de mutação atua sobre cada membro da nova geração com probabilidade de 0,05

### Desenvolvimento:

Nesta sessão será demonstrado o passo a passo para a solução do problema proposto, incluindo:
- A definição da função de aptidão  
- A definição das coordenadas para as 20 cidades
- A geração da população inicial
- A criação da Roleta
- A obtenção das próximas gerações

[Aqui](src/Documentacao.html) você encontra a documentação dos métodos criados.


#### Definição da função de aptidão:

In [28]:
import random
from math import sqrt
import matplotlib.pyplot as plt

distancia_euclidiana = lambda p1, p2: sqrt(pow(coordenadas['x'][p2]-coordenadas['x'][p1], 2) 
                                           + pow(coordenadas['y'][p2]-coordenadas['y'][p1], 2))

# Esta função calcula as distâncias entre todas as cidades.
# Retorna uma lista contendo a distância de qualquer ponto até qualquer outro ponto.
def obter_distancias():
    distancias = []
    for i in range(20):
        aux = []
        for j in range(20):
            aux.append(distancia_euclidiana(i, j))
        distancias.append(aux)

    return distancias


# Esta função calcula a distância para se percorer as cidades indicadas pelo cromossomo.
# Retorna o custo total para passar por todas as cidades e retornar a origem.
def obter_distancia_total(cromossomo, distancias):
    x = 0
    for i in range(len(cromossomo)-1):
        x += distancias[cromossomo[i]][cromossomo[i+1]]
    # Adiciona a distância para retornar a cidade inicial
    x += distancias[cromossomo[-1]][cromossomo[0]]

    return x

#### Definição das coordenadas para as 20 cidades:

In [21]:
# Esta função gera um cromossomo com a ordem que as cidades devem ser percorridas e a distância total do percurso.
# Retorna uma tuple onde o primeiro elemento é a ordem em que as cidades devem ser percorridas e o segundo 
# elemento é a distância total do caminho.
def gerar_cromossomo(distancias):
    # Lista aleatória sem números duplicados
    cromossomo = random.sample(range(20), 20)
    distancia = obter_distancia_total(cromossomo, distancias)
    return (cromossomo, distancia)

# Criação das coordenadas das cidades
random_int_x = random.sample(range(0, 100), 20) # coordenada x
random_int_y = random.sample(range(0, 100), 20) # coordenada y
coordenadas = {'x': [x/100 for x in random_int_x], 'y': [y/100 for y in random_int_y]} # dict de coordenadas

#### Geração da população inicial:

In [22]:
# Criação da matriz de adjacências
distancias  = obter_distancias()  # matrix de adjacência
populacao = [gerar_cromossomo(distancias) for i in range(20)] # todos os cromossomos com a distância total do caminho

# Ordenar a população
populacao = sorted(populacao, key= lambda s: s[1])

#### Criação da Roleta:

In [29]:
# Roleta
roleta = []
for i in range(10):
    roleta += [(i)]*(10-i)

#### Obtenção das próximas gerações:

In [24]:
# Esta função faz o cruzamento entre o pai "a" e "b".
# Retorna uma lista contendo dois novos cromossomos.
def cycle(paiA, paiB):

    # Esta função realiza a troca dos genes do pai "a" e "b" indicados pelo indice.
    def trocar(index):
        aux = paiA[index]
        paiA[index] = paiB[index]
        paiB[index] = aux

    # Esta função verifica se o cromossomo pai "a" possui genes duplicados.
    def verificar_duplicados():
        for i in range(len(paiA) - 1):
            for j in range(i + 1, len(paiA)):
                if paiA[i] == paiA[j]:
                    return True
        return False

    # Esta função escolhe aleatoriamente um gene para ser trocado entre o pai "a" e "b" e realiza a troca.
    # Retorna o índice do gene que foi efetuado a troca.
    def passo1():
        index = random.choice(list(range(20)))
        trocar(index)
        return index

    # Esta função realiza uma troca de genes entre os cromossomos pai "a" e "b", o índice da troca é definido
    # pelo gene repetido no pai "a".
    # Retorna o índice do gene que foi efetuado a troca.
    def passo2(index):
        if paiA[index] == paiB[index]:
            return
        for idx, a in enumerate(paiA):
            if paiA[index] == a and idx != index:
                trocar(idx)
                index = idx
                break
        return idx

    # Esta função realiza a troca de genes até que o cromossomo pai "a" não possua mais genes repetidos.
    # Retorna dois novos cromossomos que surgiram a partir do cruzamento do pai "a" e "b". 
    def passo3(index):
    # Enquanto o pai "a" tiver genes repetidos, troca genes com o pai "b"
        while verificar_duplicados():
            index = passo2(index)
        return paiA, paiB

    idx = passo1()
    idx = passo2(idx)
    return passo3(idx)


# Esta função realiza a mutação no cromossomo filho. A mutação consiste em inverter o valores de dois índices 
# selecionados aleatoriamente.
# Retorna o cromossomo mutado
def mutacao(filho):
    # Escolhe dois genes para inverter no cromossomo
    lista = list(range(19))
    index_1 = random.choice(lista)
    lista.remove(index_1)
    index_2 = random.choice(lista)

    aux = filho[index_1]
    [index_1] = filho[index_2]
    filho[index_2] = aux

    return filho

# Está função realiza a execução do algoritmo genético a partir de uma população passada por parâmetro.
# Retorna uma lista com a nova população, a quantidade de individuos gerados e a quantidade dos individuos
# que foram mutados.
def gerar_geracao(populacao):
    qtdMutacoes = 0
    populacao_gerada = 0

    # Roleta - Sortear
    sorteadosA = [random.choice(roleta) for i in range(5)]
    sorteadosB = [random.choice(roleta) for i in range(5)]

    filhos = []
    for paiA, paiB in zip(sorteadosA, sorteadosB):
        # Realiza o cruzamento
        filho_a, filho_b = cycle(populacao[paiA][0], populacao[paiB][0])

        populacao_gerada += 2

        # Aplica mutação se pm for menor ou igual a 5
        pm = random.randint(1, 100)
        if pm <= 5:
            filho_a = mutacao(filho_a)
            qtdMutacoes += 1

        pm = random.randint(1, 100)
        if pm <= 5:
            filho_b = mutacao(filho_b)
            qtdMutacoes += 1

        # Encapsula os cromossomos em uma tupla
        filho_a = (filho_a, obter_distancia_total(filho_a, distancias))
        filho_b = (filho_b, obter_distancia_total(filho_b, distancias))

        filhos.append(filho_a)
        filhos.append(filho_b)

    # Substitui os 10 últimos cromossomos pelos 10 novos filhos
    populacao = populacao[:10]
    for f in filhos:
        populacao.append(f)

    # Ordena a população com os novos cromossomos
    populacao = sorted(populacao, key= lambda s: s[1])

    return populacao, populacao_gerada, qtdMutacoes

### Resultados:

Nesta seção será demonstrado o caminho encontrado pelo algoritmo, bem como os resultados referentes para:

- O tamanho da populacao
- A taxa de mutacao 
- O número de cidades 
- O melhor custo 
- A melhor solução


In [27]:
print()
for i in populacao:
    print(i)

qtdMutacoes, pop_g = 0, 0
# Aplica o algoritmo genético com o critério de parada de 10000 repetições
for i in range(10000):
    populacao, qtdPopulacaoGerada, mutacoes = gerar_geracao(populacao)
    qtdMutacoes += mutacoes
    pop_g += qtdPopulacaoGerada

print()
for i in populacao:
    print(i)

print("Populacao inicial: " + str(len(populacao)))
print("Populacao gerada: " + str(pop_g))
print("Tamanho da população: " + str(len(populacao) + pop_g))
print("Taxa de mutação: " + str((qtdMutacoes/pop_g)*100) + "%")
print("Numero de cidades: " + str(len(populacao[0][0])))
print("Melhor custo: " + str(populacao[0][1]))
print("Melhor solução: " + str(populacao[0][0]))

#plotar
cord_x = []
cord_y = []
for city in populacao[0][0]:
    cord_x.append(coordenadas['x'][city])
    cord_y.append(coordenadas['y'][city])

ax = plt.subplot(111)
ax.set_xlabel("Coordenada X", fontsize = 10)
ax.set_ylabel("Coordenada Y", fontsize = 10)
ax.set_title("Percurso das cidades", fontsize = 15)
ax.scatter(cord_x, cord_y, s = 30, color = "black", marker = "X")


cord_x.append(cord_x[0])
cord_y.append(cord_y[0])

# Liga as cidades
plt.plot(cord_x, cord_y, color = "purple", linestyle="solid", linewidth=1)

# Exibe o nome das cidades no gráfico
for index, city in enumerate(populacao[0][0]):
    ax.annotate(city, (cord_x[index], cord_y[index]), color="r")

plt.show()


([15, 2, 10, 6, 0, 5, 7, 13, 14, 1, 12, 16, 19, 4, 8, 11, 18, 3, 17, 9], 9.021279631529943)
([0, 5, 16, 19, 9, 17, 11, 15, 7, 18, 14, 1, 4, 2, 12, 3, 13, 10, 8, 6], 9.021279631529943)
([0, 5, 16, 19, 9, 17, 11, 15, 7, 18, 14, 1, 4, 2, 12, 3, 13, 10, 8, 6], 9.69623748576032)
([15, 2, 10, 6, 0, 5, 7, 13, 14, 1, 12, 16, 19, 4, 8, 11, 18, 3, 17, 9], 9.69623748576032)
([3, 14, 15, 11, 0, 8, 18, 9, 12, 13, 5, 10, 6, 2, 4, 17, 19, 16, 1, 7], 9.821470547967488)
([3, 14, 15, 11, 0, 8, 18, 9, 12, 13, 5, 10, 6, 2, 4, 17, 19, 16, 1, 7], 9.852124771635031)
([12, 13, 0, 6, 10, 8, 16, 17, 1, 2, 15, 18, 3, 11, 14, 4, 7, 5, 9, 19], 9.88652388970349)
([0, 5, 16, 19, 9, 17, 11, 15, 7, 18, 14, 1, 4, 2, 12, 3, 13, 10, 8, 6], 9.994016697489723)
([0, 5, 16, 19, 9, 17, 11, 15, 7, 18, 14, 1, 4, 2, 12, 3, 13, 10, 8, 6], 9.994016697489723)
([0, 5, 16, 19, 9, 17, 11, 15, 7, 18, 14, 1, 4, 2, 12, 3, 13, 10, 8, 6], 9.994016697489723)
([15, 2, 10, 6, 0, 5, 7, 13, 14, 1, 12, 16, 19, 4, 8, 11, 18, 3, 17, 9], 9.9940166

TypeError: cannot unpack non-iterable int object